In [11]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import ta
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from torch.utils.data import DataLoader, random_split
from skorch import NeuralNetClassifier

In [4]:
data = yf.download(
    "SPY",
    start="1990-01-01",
    end="2021-01-01",
    auto_adjust = True,
    group_by="Ticker",
)

[*********************100%***********************]  1 of 1 completed


In [5]:
df = ta.utils.dropna(data)
df = ta.add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True)

/Users/qingweilan/.pyenv/versions/3.8.12/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/qingweilan/.pyenv/versions/3.8.12/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [6]:
def setlabel(row):
    return 1 if row['next_diff_curr'] >= 0 else 0

df['next_diff_curr'] = df['Close'].shift(-1) - df['Close']
df['label'] = df.apply(setlabel, axis=1)

df['feature_sma_diff'] = df['trend_sma_fast'] - df['trend_sma_slow']
df['feature_ema_diff'] = df['trend_ema_fast'] - df['trend_ema_slow']
df['feature_diff_bbl'] = df['Close'] - df['volatility_bbl']
df['feature_diff_bbh'] = df['volatility_bbh'] - df['Close']

In [7]:
df = df.drop(columns=['next_diff_curr'])
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'volume_adi', 'volume_obv',
       'volume_cmf', 'volume_fi', 'volume_mfi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_nvi', 'volume_vwap', 'volatility_atr',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw',
       'volatility_bbp', 'volatility_bbhi', 'volatility_bbli',
       'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
       'volatility_kcp', 'volatility_kchi', 'volatility_kcli',
       'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_ui', 'trend_macd', 'trend_macd_signal',
       'trend_macd_diff', 'trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast',
       'trend_ema_slow', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', '

In [25]:
processed = df[84:]

cols = set(df.columns)
cols.remove('label')
cols = list(cols)
X = processed[cols].values
y = processed[['label']].values.squeeze(1)

In [36]:
X = X.astype(np.float32)
y = y.astype(np.int64)

In [37]:
class StockPredictor(nn.Module):

    def __init__(self, feature_size):
        super(StockPredictor, self).__init__()
        self.m1 = nn.Sequential(
            nn.Linear(feature_size, 256),
            nn.Sigmoid(),
            nn.Linear(256, 512),
            nn.Sigmoid(),
            nn.Linear(512, 256),
            nn.Sigmoid(),
            nn.Linear(256, 128),
            nn.Sigmoid(),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        out = self.m1(x)
        return out

In [39]:
net = NeuralNetClassifier(
    module=StockPredictor,
    module__feature_size=len(cols),
    criterion=nn.CrossEntropyLoss,
    max_epochs=100,
    lr=0.0001,
    optimizer=torch.optim.Adam,
)

net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6919       0.5482        0.6893  0.2954
      2        0.6891       0.5482        0.6886  0.2668
      3        0.6887       0.5482        0.6887  0.2637
      4        0.6888       0.5482        0.6887  0.2655
      5        0.6887       0.5482        0.6887  0.2659
      6        0.6887       0.5482        0.6887  0.2627
      7        0.6887       0.5482        0.6887  0.2621
      8        0.6887       0.5482        0.6887  0.2640
      9        0.6886       0.5482        0.6887  0.2628
     10        0.6886       0.5482        0.6887  0.2625
     11        0.6885       0.5482        0.6887  0.2627
     12        0.6885       0.5482        0.6887  0.2655
     13        0.6885       0.5482        0.6887  0.2643
     14        0.6884       0.5482        0.6886  0.2626
     15        0.6884       0.5482        0.6886  0.2670
     16        0.6883       0.5

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=StockPredictor(
    (m1): Sequential(
      (0): Linear(in_features=92, out_features=256, bias=True)
      (1): Sigmoid()
      (2): Linear(in_features=256, out_features=512, bias=True)
      (3): Sigmoid()
      (4): Linear(in_features=512, out_features=256, bias=True)
      (5): Sigmoid()
      (6): Linear(in_features=256, out_features=128, bias=True)
      (7): Sigmoid()
      (8): Linear(in_features=128, out_features=2, bias=True)
      (9): Softmax(dim=1)
    )
  ),
)